## HW 6, Part 2:   Recurrent Neural Networks (RNN)  -- Extra Credit (40 points)

When it comes to model sequential data such as sentences, documents and videos, etc, the state of the art approach is to use Recurrent neural network (RNN). At each timestep, RNN takes an element (such as a word) as input, combines with past information encoded as a vector (such as all information in the sentence before this timestep), generate a new vector encoding both current input and past information, then delivers it to next timestep.

For more details about LSTM (a very popular variant of RNN), please refer to http://colah.github.io/posts/2015-08-Understanding-LSTMs/ and here is a very good video explaining RNN: https://www.youtube.com/watch?v=WCUNPb-5EYI.

### Generating text with Long Short-Term Memory Networks

RNN can be used to generate text. For more information, please read: https://karpathy.github.io/2015/05/21/rnn-effectiveness/.

The following is an example script to generate text from Nietzsche's writings.

Note: 
- At least 10 epochs are required before the generated text
starts sounding coherent, but more is better.

- It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

- If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

In [1]:
#Import necessary libraries 
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [2]:
#Get the data - available from amazon
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower() # make it all lowercase 
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 600893
total chars: 57


In [3]:
text

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to truth, have been unskilled and unseemly methods for\nwinning a woman? certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--if,\nindeed, it stands at all! for there are scoffers who maintain that it\nhas fallen, that all dogma lies on the ground--nay more, that it is at\nits last gasp. but to speak seriously, there are good grounds for hoping\nthat all dogmatizing in philosophy, whatever solemn, whatever conclusive\nand decided airs it has assumed, may have been only a noble puerilism\nand tyronism; and probably the time is at hand when it will be once\nand again understood what has actually sufficed for the basis of such\

In [4]:
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '=': 22,
 '?': 23,
 '[': 24,
 ']': 25,
 '_': 26,
 'a': 27,
 'b': 28,
 'c': 29,
 'd': 30,
 'e': 31,
 'f': 32,
 'g': 33,
 'h': 34,
 'i': 35,
 'j': 36,
 'k': 37,
 'l': 38,
 'm': 39,
 'n': 40,
 'o': 41,
 'p': 42,
 'q': 43,
 'r': 44,
 's': 45,
 't': 46,
 'u': 47,
 'v': 48,
 'w': 49,
 'x': 50,
 'y': 51,
 'z': 52,
 'ä': 53,
 'æ': 54,
 'é': 55,
 'ë': 56}

In [5]:
indices_char

{0: '\n',
 1: ' ',
 2: '!',
 3: '"',
 4: "'",
 5: '(',
 6: ')',
 7: ',',
 8: '-',
 9: '.',
 10: '0',
 11: '1',
 12: '2',
 13: '3',
 14: '4',
 15: '5',
 16: '6',
 17: '7',
 18: '8',
 19: '9',
 20: ':',
 21: ';',
 22: '=',
 23: '?',
 24: '[',
 25: ']',
 26: '_',
 27: 'a',
 28: 'b',
 29: 'c',
 30: 'd',
 31: 'e',
 32: 'f',
 33: 'g',
 34: 'h',
 35: 'i',
 36: 'j',
 37: 'k',
 38: 'l',
 39: 'm',
 40: 'n',
 41: 'o',
 42: 'p',
 43: 'q',
 44: 'r',
 45: 's',
 46: 't',
 47: 'u',
 48: 'v',
 49: 'w',
 50: 'x',
 51: 'y',
 52: 'z',
 53: 'ä',
 54: 'æ',
 55: 'é',
 56: 'ë'}

In [6]:
# Cut the text in semi-redundant sequences of maxlen characters
## Cut the text into a series of windows. 
## Each window is 40 characters
## The window moves 3 steps forward each step

maxlen = 40
step = 5
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

# Turn these sentances into one-hot encoded vectors
## For all words in the sentances, there is a one, else there is a zero in that index of the vector

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print('Done!')

nb sequences: 120171
Vectorization...
Done!


Now we have data to feed a model for text generation. Next  we build a LSTM model to fit the data. Using Keras this is only few lines of code!

In [7]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print('Done!')

Build model...
Done!


In [8]:
maxlen, len(chars)

(40, 57)

In [9]:
x.shape

(120171, 40, 57)

In [10]:
y.shape

(120171, 57)

In [12]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.ma.log(preds)
    preds = preds.filled(0)
    preds = preds / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # clear_output()
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % (epoch+1))

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.4, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

# Training
-  Each epoch takes up to 1 minute or so on a CPU (an epoch took 30 seconds for my PC)
-  Recall that training on at least 20 epochs will give intelligible results 
-  So you're gonna have to let this run for a while (if ETA per epoch is bigger than 5 minutes on your machine, you can reduce the number of epochs)

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
934/939 [============================>.] - ETA: 0s - loss: 2.1064
----- Generating text after Epoch: 1
----- diversity: 0.2
----- Generating with seed: "stand in need of the most effective mean"
stand in need of the most effective mean to be in the most to be not the propesse of the mose to be to be not the for have the most and the dought and the for the self-that the self-the most to be whole in the something to be whole the perhars and in the most to be in the profice to be whole and the himself and the being to be in the propession of the soul and to be the soul and the for the for our to be the soul and the propessity and 
----- diversity: 0.4
----- Generating with seed: "stand in need of the most effective mean"
stand in need of the most effective means and to be are and and the new its contided and and their the porting in the most to profice with the more and our under the manger to itself to banger to prepuritions of the provilual belighte to be in the more and all 

In [6]:
from numpy import *

## Load pre-trained model
Since it is time consuming to train this LSTM model with CPU for more epochs, we provided a pre-trained model which is trained on GPU for 100 epochs. Use the following code to check how coherency the model is.

It requires h5py packages, please install it to test the following code.

In [12]:
# build the model: a single LSTM
print('Load pre-trained model...')
from keras.models import load_model
model = load_model('shakespear200.h5')


def lstm_generate(seed, model):
    orig_seed = seed
    for diversity in [0.2, 0.3, 0.5, 1.0]:
        print('----- diversity:', diversity)
        seed = orig_seed
        generated = ''
        generated += seed
        print('----- Generating with seed: "' + seed + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(seed):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            seed = seed[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


seed = "from an anguish with which no other is t"
# seed = "thou art"
lstm_generate(seed, model)


Load pre-trained model...
----- diversity: 0.2
----- Generating with seed: "from an anguish with which no other is t"
from an anguish with which no other is the sense of the subjection of the something the sake to the sense of the present the subjection of the subjection of the more constitute of the see the particularly in the same the subjection of the master, and a such a reality and superfection of the sake of the present in the subjection of the sense of the so the subjection of the subjection of the such a present the present the so the subjectio
----- diversity: 0.3
----- Generating with seed: "from an anguish with which no other is t"
from an anguish with which no other is to the served and the serve in the consideration of the sense of the south and about the sense of the subjection of the so the conscious and the subjection of the more consciousness and such a subjection of the state of the case to the present the subjection of the serve the southerness of the see the subject

### Exercise: use LSTM to generate baby names
-  The following data set contains 8000 last names. You can download and process the name data set as follows:

```python
name_path = get_file('names.txt', origin='http://www.cs.cmu.edu/afs/cs/project/ai-repository/ai/areas/nlp/corpora/names/other/names.txt')
with io.open(name_path, encoding='utf-8') as f:
    text = f.read() # make it all lowercase 
    
text = text.split()
text = ', '.join(text)
```

Using the last name data set, answer the following questions:

1. (30 points) Train a LSTM to generate the names. How long does it take to train? How coherent does it sound? 
2. (10 points) Can you train the LSTM, but for every epoch, shuffle the order of names before call model.fit()? How long does it take to train? Does it improve the coherency?



In [2]:
name_path = get_file('names.txt', origin='http://www.cs.cmu.edu/afs/cs/project/ai-repository/ai/areas/nlp/corpora/names/other/names.txt')
with io.open(name_path, encoding='utf-8') as f:
    text = f.read() # make it all lowercase 
    text = text.lower()

text = text.split()
text = ', '.join(text)

In [3]:
text

"aaberg, aaccf, aaker, aamodt, aardema, aaren, aarika, aaron, aarons, aaronson, aartjan, aase, aasen, abad, abadie, abagael, abagail, abahri, abair, abalos, abarca, abare, abate, abbas, abbasi, abbate, abbatiello, abbe, abbett, abbey, abbi, abbie, abbitt, abbot, abbott, abboud, abbruzzese, abbs, abby, abbye, abdalla, abdallah, abdella, abdo, abdul, abdulla, abdullah, abe, abed, abee, abel, abela, abelard, abele, abeles, abell, abella, abeln, abels, abelson, abend, abendroth, aber, abercrombie, aberg, aberle, abernathy, abernethy, abert, abeyta, abie, abigael, abigail, abigale, abila, abitz, able, abler, ables, abner, abney, abood, abott, aboud, abplanalp, abra, abraham, abrahamian, abrahams, abrahamsen, abrahamson, abram, abramczyk, abramo, abramovitz, abramowicz, abramowitz, abrams, abramson, abrego, abreu, abril, abron, abruzzese, abruzzo, absher, abshier, abshire, absolom, abston, abt, abts, abu, acacia, acampora, acantha, accardi, accardo, access, accetta, accola, accomando, accoun

In [4]:
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 501788
total chars: 32


In [5]:
chars

[' ',
 "'",
 ',',
 '-',
 ';',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [6]:
maxlen = 15
step = 5
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

# Turn these sentances into one-hot encoded vectors
## For all words in the sentances, there is a one, else there is a zero in that index of the vector

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print('Done!')

nb sequences: 100355
Vectorization...
Done!


In [7]:
x.shape

(100355, 15, 32)

In [8]:
y.shape

(100355, 32)

In [9]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print('Done!')

Build model...
Done!


In [14]:
def on_epoch_end(epoch, logs):
    # clear_output()
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % (epoch+1))
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.4, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(30):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [15]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
%timeit -n1 -r1 model.fit(x, y, batch_size=128, epochs=10, callbacks=[print_callback])

Epoch 1/10
780/785 [============================>.] - ETA: 0s - loss: 0.9217
----- Generating text after Epoch: 1
----- diversity: 0.2
----- Generating with seed: "toria, victoria"
toria, victoria, vicouri, vicoro, vicoro, vic
----- diversity: 0.4
----- Generating with seed: "toria, victoria"
toria, victoria, victo, vico, vico, vico, vic
----- diversity: 0.5
----- Generating with seed: "toria, victoria"
toria, victoria, vicuto, vicson, vicson, vics
----- diversity: 1.0
----- Generating with seed: "toria, victoria"
toria, victoria, vutteid, vettenei, vettert, 
785/785 [==============================] - 11s 14ms/step - loss: 0.9221
Epoch 2/10
778/785 [============================>.] - ETA: 0s - loss: 0.8558
----- Generating text after Epoch: 2
----- diversity: 0.2
----- Generating with seed: "wbotham, rowden"
wbotham, rowden, rowder, rowder, rowder, rowd
----- diversity: 0.4
----- Generating with seed: "wbotham, rowden"
wbotham, rowden, rowder, rowder, rowder, rowd
----- diversity: 0.5
-

It takes 1 minute and 30 secs to train the LSTM model (I used the google colab GPU). And it seems to make sense the output in some cases.

In [16]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.ma.log(preds)
    preds = preds.filled(0)
    preds = preds / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # clear_output()
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % (epoch+1))
    text_shuffled = text.split(",")
    random.shuffle(text_shuffled)
    text_shuffled = ','.join(map(str, text_shuffled))
    start_index = random.randint(0, len(text_shuffled) - maxlen - 1)
    for diversity in [0.2, 0.4, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text_shuffled[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(30):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [17]:
%timeit -n1 -r1 model.fit(x, y, batch_size=128, epochs=10, callbacks=[print_callback])

Epoch 1/10
785/785 [==============================] - ETA: 0s - loss: 0.5968
----- Generating text after Epoch: 1
----- diversity: 0.2
----- Generating with seed: "k, chuck, chudy"
k, chuck, chudy, chudy, chudner, chudney, chu
----- diversity: 0.4
----- Generating with seed: "k, chuck, chudy"
k, chuck, chudy, chudy, chudne, chudner, chue
----- diversity: 0.5
----- Generating with seed: "k, chuck, chudy"
k, chuck, chudy, chudyer, chue, chuel, chuell
----- diversity: 1.0
----- Generating with seed: "k, chuck, chudy"
k, chuck, chudy, chudyer, chuing, chuinngsha,
785/785 [==============================] - 9s 11ms/step - loss: 0.5968
Epoch 2/10
778/785 [============================>.] - ETA: 0s - loss: 0.5793
----- Generating text after Epoch: 2
----- diversity: 0.2
----- Generating with seed: "denberg, rodenb"
denberg, rodenberg, rodenberg, rodenberg, rod
----- diversity: 0.4
----- Generating with seed: "denberg, rodenb"
denberg, rodenbach, rodenbaugh, rodenberg, ro
----- diversity: 0.5
--

It takes 2 minutes and 20 seconds to train the LSTM (using GPU with google colab)
In this case, the output does not make as much sense as before.